In [5]:
!pip install transformers 

In [6]:
import torch
import random
random.seed(42)

In [7]:
train_text_batch = ["この電話は、かなり不便だ。", "この電話は使いやすい。","このラーメン屋のラーメンはまずい","リゲルに行きたい","あいつに負けたぜ"]
train_labels = [1, 0, 1, 0, 1]

#予め用意した教師データ(csv)を読み込む
#csvファイルを予めアップロードしておくこと

import pandas as pd

tweet_df = pd.read_csv("../../centimental_analysis/Tweetdata_utf-16.csv")
tweet_df["ネガ"] = pd.to_numeric(tweet_df["ネガ"] , downcast='integer')
tweet_df["ポジ"] = pd.to_numeric(tweet_df["ポジ"] , downcast='integer')

#ネガ、ネガ以外で1000件ずつ取得する
#
nega_df = tweet_df[tweet_df["ネガ"] == 1.0]
nega_df = nega_df[0:1000]
nega_df

no_nega_df = tweet_df[tweet_df["ネガ"] == 0.0]
no_nega_df = no_nega_df[0:1000]
no_nega_df
tweet_df = pd.concat([nega_df, no_nega_df])
tweet_df = tweet_df.reset_index(drop=True)
tweet_df

#教師データ用に整形
train_text_batch = nega_df['ツイート'].values.tolist()
train_text_batch.extend(tweet_df['ツイート'].values.tolist())

train_labels = nega_df['ネガ'].values.tolist()
train_labels.extend(tweet_df['ネガ'].values.tolist())


#本来は、ここで前処理を実施するべきだが、処理の流れを追っていく目的があるため省く

In [9]:
import collections
#ここでラベルの割合を確認する
c = collections.Counter(train_labels)
c

Counter({1: 3, 0: 2})

In [10]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', return_dict=True)

# 訓練モードに変更
model.train()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [11]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5) # AdamWオプティマイザ

In [12]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


#トークンに分割する(tokenizer内部でmecab使った形態素の処理をしている)
#bertでは、文頭に[CLS]、分の終わりに[SEP]が必要だが、以下の処理では、文章を入れるだけで[CLS]、[SEP]を設定してくれる
#さらに、paddingをTrueにすることで、文字列が最も長い文に合わせて配列の大きさを調整する処理もやってくれるので便利だ
encoding = tokenizer(train_text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

encoding

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    2,    70,   327,   735,     9,     6,  3403,   269,  2172,    75,
             8,     3],
        [    2,    70,   327,   735,     9,   196, 17989, 25372,     8,     3,
             0,     0],
        [    2,    70,   422,  2723,  1341,     5,   422,  2723, 28450, 26453,
         28457,     3],
        [    2,   182,  6111, 28446,    77, 22815, 28457,     3,     0,     0,
             0,     0],
        [    2,  4669, 28532, 28446,  1083,  1749, 28447, 29695,     3,     0,
             0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1,

In [ ]:
tokenized_text = tokenizer.tokenize(train_text_batch[0])
print(tokenized_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

print(tokenizer.convert_tokens_to_ids(["[CLS]"]))
print(tokenizer.convert_tokens_to_ids(["[SEP]"]))

In [13]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from sklearn.model_selection import train_test_split

#tensor型に変換
input_ids = torch.tensor(input_ids)
attention_mask = torch.tensor(attention_mask)
train_labels = torch.tensor(train_labels)

# データセットクラスの作成
# なんでTensorDataset使ってデータを分割しているかって？
# 分割しなくても良いけど、その場合、メモリが16G程度だとすぐにRAMがいっぱいになてしまうんだ
dataset = TensorDataset(input_ids, attention_mask, train_labels)


#トレーニングデータ、評価データを9:1の割合で分割する
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを訓練用と検証用に分ける
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(train_dataset[0])
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100,
#                                                    stratify=y)

print('訓練データ数：{}'.format(train_size))
print('検証データ数:　{} '.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

(tensor([    2,  4669, 28532, 28446,  1083,  1749, 28447, 29695,     3,     0,
            0,     0]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]), tensor(1))
訓練データ数：4
検証データ数:　1 


<ipython-input-13-53925c403083>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids)
<ipython-input-13-53925c403083>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask)


In [14]:
input_ids

tensor([[    2,    70,   327,   735,     9,     6,  3403,   269,  2172,    75,
             8,     3],
        [    2,    70,   327,   735,     9,   196, 17989, 25372,     8,     3,
             0,     0],
        [    2,    70,   422,  2723,  1341,     5,   422,  2723, 28450, 26453,
         28457,     3],
        [    2,   182,  6111, 28446,    77, 22815, 28457,     3,     0,     0,
             0,     0],
        [    2,  4669, 28532, 28446,  1083,  1749, 28447, 29695,     3,     0,
             0,     0]])

In [15]:
# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0


    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        #outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #log_str = "loss値は「%s」" % str(loss.item())
        #print(log_str)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 検証モードに切り替え
    val_loss = 0
    with torch.no_grad(): # 訓練じゃないので勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            val_loss += loss.item()
    return val_loss

In [16]:
# 学習の実行
max_epoch = 20
train_loss_ = []
test_loss_ = []

#エポック数の設定
for epoch in range(max_epoch):
    train_ = train(model)
    #test_ = train(model)
    train_loss_.append(train_)
    #test_loss_.append(test_)

    print(train_)

NameError: name 'device' is not defined

In [ ]:
model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        print(f'出力:{preds}')

In [ ]:
# データの比較しやすい様にpd.dataframeへ整形する
import pandas as pd
import numpy as np
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])

## np.argmaxで大き方を予測の正解とする
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label']) #予測したラベル
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label']) #正解のラベル

#！評価としては本来は、AUCなど使ったほうが良い
accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

In [ ]:
accuracy_df.to_csv("accuracy_df.csv")

In [ ]:
#作成したモデルを保存する

#参考サイト
#https://tzmi.hatenablog.com/entry/2020/03/05/222813

#GPUバージョンで保存
model_path = 'gpu_model.pth'
torch.save(model.state_dict(), model_path)

#念の為読み込めることを確認しておく
model.load_state_dict(torch.load(model_path))



#CPUバージョンでも保存(ローカルのPCでも使えるようにしておく)
model_path = 'cpu_model.pth'
torch.save(model.to('cpu').state_dict(), model_path)